# File Summary:

This file performs EDA on the data from Reddit using the pushshift api, see notebook 3a.

In this file, I combined the files created in notebook 3a.  To account for the overlapping information in each fild, I looked up the timestamp of the last entry for each file.  I then found in which entry that timestamp appears in the next file, and used the index of it when concatenating all the files so that duplicates did not occur.

Note that I checked how often multiple entries have the same timestamp.  I found 20 entries with duplicate timestamps out of 16000 entries in the first file.  So I didn't worry about the risk that the last entry in one file, would not be the same entry as the first time that the timestamp of that last entry appears in the next file.

After combining the entire republican and entire democrat data and removing duplicates, I then selected key features that I found were similar in both.  I based these features on ones that I used in the analysis on the data downloaded using the PRAW api, as well as features that appeared to generally have information.  

Finally, I filtered the both republican and democrat datasets so that I only had the key features in the two dataframes, and I combined these into one and saved it (reps_and_dems.csv) for further analysis in other notebooks.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# display all columns
# from https://towardsdatascience.com/how-to-show-all-columns-rows-of-a-pandas-dataframe-c49d4507fcf

pd.set_option('display.max_columns', None)

In [3]:
dems1 = pd.read_csv('../data/dems.csv')
dems2 = pd.read_csv('../data/dems2.csv')
dems3 = pd.read_csv('../data/dems3.csv')
dems4 = pd.read_csv('../data/dems4.csv')
dems5 = pd.read_csv('../data/dems5.csv')
dems6 = pd.read_csv('../data/dems6.csv')
dems7 = pd.read_csv('../data/dems7.csv')

/Users/jenniferwilliamson/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8,9,25,41,57,60,63,70,71,74,75,76,77,78,79,82,83,84,86,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/jenniferwilliamson/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,3,5,9,49,73) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/jenniferwilliamson/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,5,6,7,8,12,14,15,16,17,18,20,22,23,25,26,28,29,33,35,40,41,46,50,60,61,62,64,68,69,70,71,73,75,76,77,78,79,80,81,82) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_

In [4]:
reps1 = pd.read_csv('../data/reps.csv')
reps2 = pd.read_csv('../data/reps2.csv')
reps3 = pd.read_csv('../data/reps3.csv')
reps4 = pd.read_csv('../data/reps4.csv')
reps5 = pd.read_csv('../data/reps5.csv')
reps6 = pd.read_csv('../data/reps6.csv')
reps7 = pd.read_csv('../data/reps7.csv')

C:\Users\jennw\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3,8,9,51,57,60,65,73,74,75,77,79,82,83,84,86,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\jennw\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,9,43) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\jennw\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2,4,6,7,8,12,14,15,16,17,18,20,22,23,24,28,29,33,35,44,45,50,56,59,60,62,63,64,67,69,72,73,75,76,78,79,80,81,82,84,86,88,89,90,91,92) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
dems1.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,awarders,can_mod_post,contest_mode,created_utc,domain,full_link,gildings,id,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,link_flair_background_color,link_flair_richtext,link_flair_text_color,link_flair_type,locked,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,post_hint,preview,pwls,removed_by_category,retrieved_on,score,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,thumbnail,thumbnail_height,thumbnail_width,title,total_awards_received,treatment_tags,upvote_ratio,url,url_overridden_by_dest,whitelist_status,wls,link_flair_template_id,link_flair_text,media,media_embed,secure_media,secure_media_embed,link_flair_css_class,is_gallery,author_cakeday,author_flair_background_color,author_flair_text_color,crosspost_parent,crosspost_parent_list,distinguished,banned_by,gallery_data,media_metadata,edited,gilded,collections,steward_reports,removed_by,updated_utc,og_description,og_title
0,[],False,SP4C3B4S3,NaN,[],NaN,text,t2_75st1vsp,False,False,[],False,False,1602943707,i.redd.it,https://www.reddit.com/r/democrats/comments/jc...,{},jcvqal,False,False,False,True,False,False,False,NaN,[],dark,text,False,False,True,2,0,False,all_ads,/r/democrats/comments/jcvqal/happy_halloween/,False,image,"{'enabled': True, 'images': [{'id': 'MShgfz739...",6,automod_filtered,1602943718,1,NaN,True,False,False,democrats,t5_2qn70,146909,public,https://b.thumbs.redditmedia.com/vtKtSn_8Vhupk...,111.0,140.0,Happy Halloween!,0,[],1.0,https://i.redd.it/jo3xacq3xnt51.jpg,https://i.redd.it/jo3xacq3xnt51.jpg,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Singularitytracker,NaN,[],NaN,text,t2_5ny3j53h,False,False,[],False,False,1602945905,i.redd.it,https://www.reddit.com/r/democrats/comments/jc...,{},jcwcnk,False,False,False,True,False,False,False,NaN,[],dark,text,False,False,True,2,0,False,all_ads,/r/democrats/comments/jcwcnk/someone_i_follow_...,False,image,"{'enabled': True, 'images': [{'id': 'MfZjwPw4T...",6,automod_filtered,1602945916,1,NaN,True,False,False,democrats,t5_2qn70,146911,public,https://b.thumbs.redditmedia.com/ikAchV1hvSmAp...,34.0,140.0,someone I follow pointed out that Russia today...,0,[],1.0,https://i.redd.it/yg8h7i2j3ot51.png,https://i.redd.it/yg8h7i2j3ot51.png,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,1000000students,NaN,[],NaN,text,t2_7vedo,False,False,[],False,False,1602945922,self.democrats,https://www.reddit.com/r/democrats/comments/jc...,{},jcwctt,False,False,False,False,False,True,False,#646d73,"[{'e': 'text', 't': '📉 Economy'}]",light,richtext,False,False,True,0,0,False,all_ads,/r/democrats/comments/jcwctt/how_donald_trump_...,False,self,"{'enabled': False, 'images': [{'id': '2LmpJAho...",6,moderator,1602945932,1,[removed],True,False,False,democrats,t5_2qn70,146911,public,self,NaN,NaN,HOW DONALD TRUMP AND THE REPUBLICAN PARTY SCRE...,0,[],1.0,https://www.reddit.com/r/democrats/comments/jc...,NaN,all_ads,6,a8f6a1ce-dd58-11ea-84d2-0e5d79fe92b3,📉 Economy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,hornet7777,NaN,[],NaN,text,t2_z3ajl,False,False,[],False,False,1602946014,borntorunthenumbers.com,https://www.reddit.com/r/democrats/comments/jc...,{},jcwdt1,False,False,False,False,False,False,False,NaN,[],dark,text,False,False,True,0,0,False,all_ads,/r/democrats/comments/jcwdt1/btrtn_ten_days_in...,False,NaN,NaN,6,moderator,1602946025,1,NaN,True,False,False,democrats,t5_2qn70,146911,public,default,NaN,NaN,BTRTN: Ten Days in November... Would You Make ...,0,[],1.0,http://www.borntorunthenumbers.com/2020/10/bt

In [6]:
reps1.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,awarders,can_mod_post,contest_mode,created_utc,domain,full_link,gildings,id,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,link_flair_background_color,link_flair_richtext,link_flair_text_color,link_flair_type,locked,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,post_hint,preview,pwls,retrieved_on,score,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,total_awards_received,treatment_tags,upvote_ratio,url,url_overridden_by_dest,whitelist_status,wls,crosspost_parent,crosspost_parent_list,removed_by_category,media,media_embed,secure_media,secure_media_embed,author_flair_background_color,author_flair_text_color,link_flair_text,gallery_data,is_gallery,media_metadata,author_flair_template_id,banned_by,author_cakeday,link_flair_template_id,edited,gilded,distinguished,steward_reports,removed_by,updated_utc,og_description,og_title
0,[],False,Foubar,NaN,[],NaN,text,t2_tpsej,False,True,[],False,False,1602943321,washingtonexaminer.com,https://www.reddit.com/r/Republican/comments/j...,{},jcvmgy,True,False,False,False,True,False,False,NaN,[],dark,text,False,False,True,33,0,False,all_ads,/r/Republican/comments/jcvmgy/biden_says_he_wi...,False,link,"{'enabled': False, 'images': [{'id': 'K32vhhq5...",6,1602943333,1,NaN,True,False,False,Republican,t5_2qndt,153016,public,confidence,https://b.thumbs.redditmedia.com/W9EQwrbw04SrL...,73.0,140.0,"Biden says he will raise taxes immediately, ev...",0,[],1.0,https://www.washingtonexaminer.com/news/biden-...,https://www.washingtonexaminer.com/news/biden-...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Dan-In-SC,NaN,[],NaN,text,t2_1o0d4hsp,False,True,[],False,False,1602943857,toptradeguru.com,https://www.reddit.com/r/Republican/comments/j...,{},jcvrrg,True,False,False,False,True,False,False,NaN,[],dark,text,False,False,True,6,0,False,all_ads,/r/Republican/comments/jcvrrg/duplicate_ballot...,False,link,"{'enabled': False, 'images': [{'id': 'O2gaCpau...",6,1602943869,1,NaN,True,False,False,Republican,t5_2qndt,153017,public,confidence,https://b.thumbs.redditmedia.com/HUzTU-6BhUBA7...,140.0,140.0,Duplicate Ballots Applications Flood Pennsylva...,0,[],1.0,https://www.toptradeguru.com/news/duplicate-ba...,https://www.toptradeguru.com/news/duplicate-ba...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Foubar,NaN,[],NaN,text,t2_tpsej,False,True,[],False,False,1602943877,hotair.com,https://www.reddit.com/r/Republican/comments/j...,{},jcvryj,True,False,False,False,True,False,False,NaN,[],dark,text,False,False,True,2,0,False,all_ads,/r/Republican/comments/jcvryj/the_associated_p...,False,link,"{'enabled': False, 'images': [{'id': 'Ladhl-zU...",6,1602943889,1,NaN,True,False,False,Republican,t5_2qndt,153017,public,confidence,https://b.thumbs.redditmedia.com/RqNnebaVh7lcF...,80.0,140.0,The Associated Press finally talks about the H...,0,[],1.0,https://hotair.com/archives/jazz-shaw/2020/10/...,https://hotair.com/archives/jazz-shaw/2020/10/...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,Dan-In-SC,NaN,[],NaN,text,t2_1o0d4hsp,False,True,[],False,False,1602944546,nypost.com,https://www.reddit.com/r/Republican/comments/j...,{},jcvymk,True,False,False,False,True,False,False,NaN,[],dark,text,False,False,True,12,0,False,all_ads,/r/Republican/comments/jcvymk/kristen_welker_u...,False,link,"{'enabled': False, 'images': [{'id': 'U1JzUPDz...",6,1602944559,1,NaN,True,False,False,Republican,t5_2qndt,153017,public,confide

# Combine the 7 data frames that I downloaded 

## there's some overlap, so I found where the timestamp for the last entry of the previous dataframe starts on the next data frame, so that I can only have one entry for each.  There's a small chance this could result in multiple entries (I found 20 entries with duplicate times stamps out of 16000 earlier, so it's highly unlikely.

In [7]:
def list_starts(list_of_dfs):
    
    list_of_indices = []
  
    for i in range(len(list_of_dfs)-1):
        df1 = list_of_dfs[i]
        df2 = list_of_dfs[i+1]
        
        last_utc = df1['created_utc'].tail(1).iloc[0]
        
        start_new_index = df2['created_utc'][df2['created_utc'] == last_utc].index.values + 1 
        
        #start_new_index = start_new_index[0]
        
        #print(f'The last utc of the {i} dataframe is {last_utc}.')
        #print(f'So the start index of the {i+1} dataframe should be {start_new_index}.')
        
        list_of_indices.append(start_new_index)
        
    return list_of_indices


In [8]:
dems_list = [dems1, dems2, dems3, dems4, dems5, dems6, dems7]
reps_list = [reps1, reps2, reps3, reps4, reps5, reps6, reps7]

In [9]:
dl = list_starts(dems_list)
dl

[array([], dtype=int64),
 array([870], dtype=int64),
 array([833], dtype=int64),
 array([50], dtype=int64),
 array([123], dtype=int64),
 array([34], dtype=int64)]

In [10]:
rl = list_starts(reps_list)
rl

[array([], dtype=int64),
 array([351], dtype=int64),
 array([598], dtype=int64),
 array([55], dtype=int64),
 array([64], dtype=int64),
 array([196], dtype=int64)]

In [11]:
all_dems = pd.concat([dems1, dems2, dems3[dl[1][0]:], dems4[dl[2][0]:],dems5[dl[3][0]:],dems6[dl[4][0]:],dems7[dl[5][0]:]])
all_dems.shape

(123668, 99)

In [12]:
all_reps = pd.concat([reps1, reps2, reps3[rl[1][0]:], reps4[rl[2][0]:], reps5[rl[3][0]:], reps6[rl[4][0]:], reps7[rl[5][0]:]])
all_reps.shape

(90351, 107)

# Reduce dataframes to only features we're interested in

In [13]:
FEATURES = ['author','author_fullname','created_utc','domain','full_link','id','num_comments','permalink','score','subreddit','title','url']

all_dems_F = all_dems[FEATURES]
all_reps_F = all_reps[FEATURES]

In [14]:
all_dems_F.shape

(123668, 12)

In [15]:
all_reps_F.shape

(90351, 12)

In [16]:
all_dems_F.isnull().sum()

author                 0
author_fullname    24401
created_utc            0
domain                 0
full_link              0
id                     0
num_comments           0
permalink              0
score                  0
subreddit              0
title                  0
url                    0
dtype: int64

In [17]:
all_reps_F.isnull().sum()

author                 0
author_fullname    15423
created_utc            0
domain                 3
full_link              0
id                     0
num_comments           0
permalink              0
score                  0
subreddit              0
title                  0
url                    0
dtype: int64

In [18]:
all_F = pd.concat([all_dems_F,all_reps_F])

In [19]:
all_F.head()

,author,author_fullname,created_utc,domain,full_link,id,num_comments,permalink,score,subreddit,title,url
0,SP4C3B4S3,t2_75st1vsp,1602943707,i.redd.it,https://www.reddit.com/r/democrats/comments/jc...,jcvqal,2,/r/democrats/comments/jcvqal/happy_halloween/,1,democrats,Happy Halloween!,https://i.redd.it/jo3xacq3xnt51.jpg
1,Singularitytracker,t2_5ny3j53h,1602945905,i.redd.it,https://www.reddit.com/r/democrats/comments/jc...,jcwcnk,2,/r/democrats/comments/jcwcnk/someone_i_follow_...,1,democrats,someone I follow pointed out that Russia today...,https://i.redd.it/yg8h7i2j3ot51.png
2,1000000students,t2_7vedo,1602945922,self.democrats,https://www.reddit.com/r/democrats/comments/jc...,jcwctt,0,/r/democrats/comments/jcwctt/how_donald_trump_...,1,democrats,HOW DONALD TRUMP AND THE REPUBLICAN PARTY SCRE...,https://www.reddit.com/r/democrats/comments/jc...
3,hornet7777,t2_z3ajl,1602946014,borntorunthenumbers.com,https://www.reddit.com/r/democrats/comments/jc...,jcwdt1,0,/r/democrats/comments/jcwdt1/btrtn_ten_days_in...,1,democrats,BTRTN: Ten Days in November... Would You Make ...,http://www.borntorunthenumbers.com/2020/10/btr...
4,GodverdommeCoffee,t2_84twh,1602946028,youtu.be,https://www.reddit.com/r/democrats/comments/jc...,jcwdy3,0,/r/democrats/comments/jcwdy3/never_forget_that...,1,democrats,Never forget that long after he's gone our chi...,https://youtu.be/TeOePoa4z-c


In [20]:
clean_FEATURES = ['created_utc','id','num_comments','score','subreddit','title','url']

all_clean = all_F[clean_FEATURES]

In [21]:
all_clean.to_csv('../data/reps_and_dems.csv', index=False)